<a href="https://colab.research.google.com/github/csinguva/Installation/blob/master/agribot_multicalls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import os
import seaborn as sns
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.metrics import classification_report
import numpy
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import re
from nltk.corpus import stopwords
import keras 
import numpy as np
from keras.preprocessing.text import Tokenizer
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, Dropout, Embedding, LSTM, Flatten
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
plt.style.use('ggplot')
%matplotlib inline
from sklearn.model_selection import train_test_split

In [0]:
from google.colab import auth
auth.authenticate_user()

In [22]:
YOUR_BUCKET_FULL_PATH = 'gs://agribot'
print('***** Bucket Sub directory: {} *****'.format(YOUR_BUCKET_FULL_PATH))
!gsutil ls -lr $YOUR_BUCKET_FULL_PATH


***** Bucket Sub directory: gs://agribot *****
    592638  2019-03-31T22:03:08Z  gs://agribot/3434101.csv
     20275  2019-03-31T21:44:17Z  gs://agribot/agribot-binary-classifiers.ipynb
     19497  2019-03-31T21:44:18Z  gs://agribot/agribot-fileprocesser.ipynb
     33695  2019-03-31T21:44:17Z  gs://agribot/agribot-multi-classifiers.ipynb
 151528584  2019-03-31T21:49:23Z  gs://agribot/weights-simple.hdf5
TOTAL: 5 objects, 152194689 bytes (145.14 MiB)


In [30]:
!gsutil cp gs://agribot/3434101.csv /content/3434101.csv

Copying gs://agribot/3434101.csv...
/ [1 files][578.8 KiB/578.8 KiB]                                                
Operation completed over 1 objects/578.8 KiB.                                    


In [31]:
ls

3434101.csv  adc.json  data  sample_data/


In [0]:
df = pd.read_csv('/content/3434101.csv', lineterminator='\n')

In [33]:
df.head()

,Unnamed: 0,Season,Sector,Category,Crop,QueryType,QueryText,KCCAns,StateName,DistrictName,BlockName,CreatedOn
0,0,JAYAD,AGRICULTURE,Cereals,Paddy (Dhan),Weather,mosam ki jankari,1-7August tak mosam me dhup or badal bana rahe...,UTTAR PRADESH,SITAPUR,RAMPUR MATHURA ...,2017-08-01 06:03:58.583
1,1,JAYAD,AGRICULTURE,Others,Others,Weather,weather information?,pure hafte barish hone ki sambhavana hai,UTTAR PRADESH,SITAPUR,LAHARPUR ...,2017-08-01 06:15:10.420
2,2,KHARIF,AGRICULTURE,Others,Others,Weather,weather information.,aj baadal chhaye rahege barish hone ki sambhav...,UTTAR PRADESH,SITAPUR,SAKRAN ...,2017-08-01 06:15:15.703
3,3,KHARIF,AGRICULTURE,Cereals,Paddy (Dhan),\tPlant Protection\t,stem borer in paddy ?,spray Cartap Hydrochloride 50% SP 400 g /acre ...,UTTAR PRADESH,SITAPUR,GONDLAMAU ...,2017-08-01 06:24:23.470
4,4,KHARIF,AGRICULTURE,Others,Others,Weather,farmer asking weather information ?,Some rainfall found in your area in 1 august ...,UTTAR PRADESH,SITAPUR,SIDHAULI ...,2017-08-01 06:58:23.267


In [34]:
# Experiment 1 - Use query Text and Query Type
exp1_xy = df.copy()
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = str (text)
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    text = str (text)
    return text    

def str_len(text):
    return len(text)

exp1_xy['QueryText'] = exp1_xy['QueryText'].apply(clean_text)
exp1_xy['QueryLen'] = exp1_xy['QueryText'].apply(str_len)
exp1_xy['QueryType'] = exp1_xy['QueryType'].apply(clean_text)

print(exp1_xy.columns)
exp1_xy = exp1_xy.drop(['Unnamed: 0','Category', 'Crop', 'KCCAns','Season','Sector','StateName', 'DistrictName', 'BlockName', 'CreatedOn','Microsoft Office User                                 '], axis=1)
print("---------------------Dropped all columns except Q Type and Q text---------------------")
print(exp1_xy.columns)

LookupError: ignored